# LIBRARIES

In [1]:
#run only once, 
!pip install ghhops-server flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.7 MB 19.2 MB/s 
  Created wheel for rhino3dm: filename=rhino3dm-7.14.2-cp37-cp37m-linux_x86_64.whl size=4995204 sha256=f3c66a7c68e3cac1453d477c85871bd5b654ec082df9fd4ca7bb4ab59a1e6c6a
  Stored in directory: /root/.cache/pip/wheels/54/fd/27/c1c91b5631a9dcb2291a8541117b0040fe461756f9c77430fc
Successfully built rhino3dm


In [2]:
# Import standard Libraries
import pandas as pd
import seaborn as sns
import altair as alt
import tensorflow as tf
import matplotlib.pyplot as plt
import joblib
from flask import Flask
from flask_ngrok import run_with_ngrok
import ghhops_server as hs
import numpy
import rhino3dm
from tensorflow import keras

sns.set(rc={'figure.figsize':(10,10)})
print("imports ok")

imports ok


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# HOPS

In [ ]:
/content/gdrive/MyDrive/ColabNotebooks/DataEncoding/atrium_SDA+ASE_regression_analysis/Iteration3_2933samples_ASE+SDA+VIEW/atrium_SDA+ASE_v3_2933_samples.csv

In [4]:
# 1 --- import model

path = "/content/gdrive/MyDrive/ColabNotebooks/DataEncoding/atrium_SDA+ASE_regression_analysis/Iteration3_2933samples_ASE+SDA+VIEW/atriumModel1.h5"
model = tf.keras.models.load_model(path)

# 2 --- set up scaler variables

scalerX = joblib.load("/content/gdrive/MyDrive/ColabNotebooks/DataEncoding/atrium_SDA+ASE_regression_analysis/Iteration3_2933samples_ASE+SDA+VIEW/scalerXAtoB.save")
scalerY = joblib.load("/content/gdrive/MyDrive/ColabNotebooks/DataEncoding/atrium_SDA+ASE_regression_analysis/Iteration3_2933samples_ASE+SDA+VIEW/scalerYAtoB.save")

[DEBUG] Creating converter from 3 to 5


In [5]:



def myMLPredictions(typicalFacadeGlazingRatio, typicalFacadeWindowWidth, groundFacadeGlazingRatio, groundFacadeWindowWidth, buildingDepth, balconyWidth, atriumArea, aspectRatio, groundFloorHeight, typicalfloorHeight, numberLevels):

    # Create Input Array
    new_data = pd.DataFrame([[typicalFacadeGlazingRatio, typicalFacadeWindowWidth, groundFacadeGlazingRatio, groundFacadeWindowWidth, buildingDepth, balconyWidth, atriumArea, aspectRatio, groundFloorHeight, typicalfloorHeight, numberLevels]])

    # Scale Input to match the same format from the trained model
    scaled_input = scalerX.transform(new_data)

    # Generate Predictions from Scaled Input using model.predict
    #https://www.tensorflow.org/api_docs/python/tf/keras/Model

    # 01// predict
    out = model.predict(scaled_input)
    #02 // inverse scaling
    predictions = scalerY.inverse_transform(out)

    
    #Flatten predictions list to be readable for hops
    pred_list = predictions.tolist()
    flat_list = []

    for i in pred_list:
        flat_list += i

    # Set each prediction value to a variable
    groundSDA= flat_list[0]
    firstSDA= flat_list[1]
    groundASE= flat_list[2]
    firstASE= flat_list[3]
    groundViewPercentage= flat_list[4]
    firstViewPercentage= flat_list[5]


    return groundSDA, firstSDA, groundASE, firstASE, groundViewPercentage, firstViewPercentage



In [6]:
app = Flask(__name__)
#using ngrok library to stream data
run_with_ngrok(app) 
hops = hs.Hops(app)

# -- ATOB MODEL

@hops.component(
    "/myAtriumDaylighting",
    name="Atrium Daylighting Analysis",
    description="Environmental analysis for atriums",
    inputs=[
        hs.HopsNumber("typicalFacadeGlazingRatio", "typicalFacadeGlazingRatio", "glazing ratio for typical floors"),
        hs.HopsNumber("typicalFacadeWindowWidth", "typicalFacadeWindowWidth", "window width for typical floor"),
        hs.HopsNumber("groundFacadeGlazingRatio", "groundFacadeGlazingRatio", "glazing ratio for ground floor"),
        hs.HopsNumber("groundFacadeWindowWidth", "groundFacadeWindowWidth", "window width for ground floor"),
        hs.HopsNumber("buildingDepth", "buildingDepth", "depth of building floorplate"),
        hs.HopsNumber("balconyWidth", "balconyWidth", "width of inner atrium balcony"),
        hs.HopsNumber("atriumArea", "atriumArea", "area of atrium"),
        hs.HopsNumber("aspectRatio", "aspectRatio", "aspect ratio of building"),
        hs.HopsNumber("groundFloorHeight", "groundFloorHeight", "floor height of ground level"),
        hs.HopsNumber("typicalfloorHeight", "typicalfloorHeight", "floor height of typical level"),
        hs.HopsInteger("numberLevels", "numberLevels", "number of building levels"),
    ],
    outputs=[
        hs.HopsNumber("groundSDA"),
        hs.HopsNumber("firstSDA"),
        hs.HopsNumber("groundASE"),
        hs.HopsNumber("firstASE"),
        hs.HopsNumber("groundViewPercentage"),
        hs.HopsNumber("firstViewPercentage"),
    
    ]
)

def myMLFunctionAtoB(typicalFacadeGlazingRatio, typicalFacadeWindowWidth, groundFacadeGlazingRatio, groundFacadeWindowWidth, buildingDepth, balconyWidth, atriumArea, aspectRatio, groundFloorHeight, typicalfloorHeight, numberLevels):
    return myMLPredictions(typicalFacadeGlazingRatio, typicalFacadeWindowWidth, groundFacadeGlazingRatio, groundFacadeWindowWidth, buildingDepth, balconyWidth, atriumArea, aspectRatio, groundFloorHeight, typicalfloorHeight, numberLevels)




In [ ]:
#run the app
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


[INFO]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[DEBUG] Starting new HTTP connection (1): localhost:4040
[DEBUG] http://localhost:4040 "GET /api/tunnels HTTP/1.1" 200 791


 * Running on http://8d37-34-86-109-249.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
